In [24]:
import pandas as pd
import numpy as np
import keras
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import keras
from keras.models import Model
from keras.layers import Input, Dense
from keras.layers import ReLU, Softmax
from keras.callbacks import CSVLogger
from sklearn.tree import DecisionTreeClassifier
from sklearn import linear_model

In [25]:
train_data=pd.read_pickle("D:\gitrep\DMlab2\DM2022-Lab2-Master-main\kaggle\\trdata_withlabel.pkl")
test_data=pd.read_pickle("D:\gitrep\DMlab2\DM2022-Lab2-Master-main\kaggle\\test_data.pkl")

In [26]:
train,val=train_test_split(train_data,test_size=0.2)
train.groupby(['emotion']).count()
val.groupby(['emotion']).count()

,tweet_id,text
emotion,,
anger,7989,7989
anticipation,50041,50041
disgust,27603,27603
fear,12746,12746
joy,103165,103165
sadness,38504,38504
surprise,9850,9850
trust,41215,41215


In [27]:
BOW_500 = CountVectorizer(max_features=500, tokenizer=nltk.word_tokenize)
#
#train data
BOW_500.fit(train.text)
x_train_BOW=BOW_500.transform(train.text)
#
y_train=train['emotion']

C:\Users\楊智均\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\feature_extraction\text.py:524: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
#val data
x_val_BOW=BOW_500.fit_transform(val.text)
#
y_val=val['emotion']

In [ ]:
#test data
BOW_500.fit(test_data.text)
x_test_BOW=BOW_500.transform(test_data.text)
#

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
print( label_encoder.classes_)


def label_encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def label_decode(le, one_hot_label):
    dec = np.argmax(one_hot_label, axis=1)
    return le.inverse_transform(dec)

y_train = label_encode(label_encoder, y_train)




In [ ]:
#linearmodel
li_model=linear_model.LinearRegression()
li_model=li_model.fit(x_train_BOW,y_train)


In [ ]:
li_pre=li_model.predict(x_test_BOW)
li_pre = label_decode(label_encoder, li_pre)
output3= pd.DataFrame({'id':test_data.tweet_id , 'emotion': li_pre})
output3.to_csv('li_pre.csv', index=False)

In [ ]:
dt_model_BOW = DecisionTreeClassifier(random_state=0)
dt_model_BOW=dt_model_BOW.fit(x_train_BOW,y_train)
#train_pred_BOW = dt_model_BOW.predict(x_train_BOW)
test_pred_BOW = dt_model_BOW.predict(x_test_BOW)
test_pred_BOW = label_decode(label_encoder, test_pred_BOW)
output1 = pd.DataFrame({'id':test_data.tweet_id , 'emotion': test_pred_BOW})
output1.to_csv('DT_BOW.csv', index=False)

In [ ]:
TFIDF_500 = TfidfVectorizer(max_features=500, tokenizer=nltk.word_tokenize)
x_train_TFIDF=TFIDF_500.fit_transform(train.text)
#x_val_TFIDF=TFIDF_500.fit_transform(val.text)
dt_model_TFIDF = DecisionTreeClassifier(random_state=0)
x_test_TFIDF=TFIDF_500.fit_transform(test_data.text)
dt_model_TFID=dt_model_TFIDF.fit(x_train_TFIDF,y_train)
test_pred_TFIDF = dt_model_TFIDF.predict(x_test_TFIDF)
test_pred_TFIDF = label_decode(label_encoder, test_pred_TFIDF)
output2 = pd.DataFrame({'id':test_data.tweet_id , 'emotion': test_pred_BOW})
output2.to_csv('DT_TFIDF.csv', index=False)

In [ ]:
output2 = pd.DataFrame({'id':test_data.tweet_id , 'emotion': test_pred_BOW})
output2.to_csv('DT_TFIDF.csv', index=False)

In [ ]:
from sklearn.linear_model import LogisticRegression
lg_model=LogisticRegression(C=1000, penalty="l2")
t_pre=label_decode(label_encoder,y_train)
lg_model_BOW=lg_model.fit(x_train_BOW,t_pre)

test_pre=lg_model_BOW.predict(x_test_BOW)

output4=pd.DataFrame({'id':test_data.tweet_id , 'emotion': test_pre})
output4.to_csv('ldpre.csv', index=False)


In [ ]:
from sklearn.neural_network import MLPClassifier
NN_model = MLPClassifier(activation='tanh', solver='sgd', max_iter=100,
                             hidden_layer_sizes=(256,64,16), random_state=None, verbose=True,
                             early_stopping=False, tol=0.0001)
    
    # Training 
NN_model = NN_model.fit(x_train_BOW, y_train)
    
    # Testing
y_train_pred = NN_model.predict(x_train_BOW)
y_test_pred = NN_model.predict(x_test_BOW)
y_test_pred=label_decode(label_encoder,y_test_pred)    
output4=pd.DataFrame({'id':test_data.tweet_id , 'emotion': y_test_pred})
output4.to_csv('mlp.csv', index=False)

    
    

In [ ]:
BOW_500.get_feature_names_out()